In [38]:
import os
import re
import ftfy
import nltk
import smtplib
import numpy as np
import pandas as pd
import pickle as pkl
from pathlib import Path
from nltk import PorterStemmer
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences

In [39]:
input_dir = 'input'
model_dir = 'model'
model_str_file = 'model_structure.json'
model_weights_file = 'model_weights.h5'
cword_file = 'cword_dict.pkl'
tokenizer_file = 'tokens.pkl'

In [40]:
class_names = [
     'Negative Sentiment',
    'Positive Sentiment'   
]
print(class_names)

['Negative Sentiment', 'Positive Sentiment']


In [41]:
cList = pkl.load(open(os.path.join(input_dir,cword_file),'rb'))

In [42]:
trained_tokenizer = pkl.load(open(os.path.join(model_dir,tokenizer_file),'rb'))

In [43]:
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

In [44]:
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [45]:
def clean_review(reviews):
    cleaned_review = []
    for review in reviews:
        review = str(review)
        if re.match("(\w+:\/\/\S+)", review) == None and len(review) > 10:
            review = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", review).split())
            review = ftfy.fix_text(review)
            review = expandContractions(review)
            review = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", review).split())
            stop_words = stopwords.words('english')
            word_tokens = nltk.word_tokenize(review) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            review = ' '.join(filtered_sentence)
            review = PorterStemmer().stem(review)
            cleaned_review.append(review)
    return cleaned_review

In [46]:
def load_trained_model(model_str_path, model_wt_path):
    f = Path(model_str_path)
    model_structure = f.read_text()
    trained_model = model_from_json(model_structure)
    trained_model.load_weights(model_wt_path)
    return trained_model

In [47]:
str_path = os.path.join(model_dir,model_str_file)
wt_path = os.path.join(model_dir,model_weights_file)

In [48]:
trained_model = load_trained_model(str_path, wt_path)

In [49]:
trained_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 140, 300)          3000000   
_________________________________________________________________
dropout_26 (Dropout)         (None, 140, 300)          0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 136, 32)           48032     
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 68, 32)            0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 68, 32)            0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 64, 64)            10304     
_________________________________________________________________
max_pooling1d_20 (MaxPooling (None, 32, 64)            0         
__________

In [50]:
test_data = ["iam so happy and joyful"]

#Me: what's wrong?My girl: *looks up at me with look of depression and stress on her face**Me: same babe same. #examseason

In [51]:
print(test_data)

['iam so happy and joyful']


In [52]:
cleaned_text = clean_review(test_data)
print(cleaned_text)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Manohar/nltk_data'
    - 'C:\\Users\\Manohar\\anaconda3\\nltk_data'
    - 'C:\\Users\\Manohar\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Manohar\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Manohar\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [53]:
sequences_text_token = trained_tokenizer.texts_to_sequences(cleaned_text)
print(sequences_text_token)

NameError: name 'cleaned_text' is not defined

In [54]:
data = pad_sequences(sequences_text_token, maxlen=140)
print('Shape of data tensor:', data.shape)

NameError: name 'sequences_text_token' is not defined

In [55]:
data

NameError: name 'data' is not defined

In [56]:
def predict_result(data, model):
    result = model.predict(data)
    Y_pred = np.round(result.flatten())
    print(Y_pred)
    for i in range(len(Y_pred)):
        print("Text : ",cleaned_text[i])
        print("Result : ",class_names[int(Y_pred[i])])

In [57]:
predict_result(data=data, model= trained_model)

NameError: name 'data' is not defined

# !!! Thank You !!! ##